In [1]:
import numpy as np
import torch
import librosa
import pandas as pd
import os
import random

In [2]:
df = pd.read_excel(r'Z:\MODMA\audio_lanzhou_2015\labels.xlsx')
df.head()

,subject id,type,age,gender,education（years）,PHQ-9,CTQ-SF,LES,SSRS,GAD-7,PSQI
0,2010002,MDD,18,F,12,23,77,-143,31,18,12
1,2010004,MDD,25,F,19,12,53,-44,38,13,11
2,2010005,MDD,20,M,16,19,49,-3,28,11,5
3,2010006,MDD,42,M,16,16,59,-30,40,12,9
4,2010008,MDD,42,M,12,17,66,-71,44,18,13


In [3]:
label = []
file = []
path_dir = r'Z:\MODMA\audio_lanzhou_2015'
a = df['subject id']
b = df['type']
for i in range(len(b)):
    file.append('0'+str(a[i]))
    if b[i] == 'MDD':
        label.append(1)
    else:
        label.append(0)
len(label)

52

In [4]:
def audio_pad(audio, sr):
    n = len(audio)
    m = sr*6
    if n < m:
        audio = np.pad(audio, (0,m-n), 'constant')
    return audio

def raw_features(audio_file):
    audio, sr = librosa.load(audio_file)
    audio = audio_pad(audio, sr)
    win_length = int(100 * sr / 1000)
    hop_length = win_length // 4

    feature1 = librosa.feature.mfcc(y=audio,
                               n_fft=win_length, 
                               hop_length=hop_length).T + 1e-12
    feature2 = librosa.feature.zero_crossing_rate(y=audio,
                                   frame_length=win_length, 
                                   hop_length=hop_length).T + 1e-12
    feature3 = librosa.feature.chroma_cqt(y=audio,
                                         sr = sr,
                                         hop_length=hop_length).T + 1e-12
#     if feature1.shape[0]==feature2.shape[0]==feature3.shape[0]:
    feature = np.concatenate([feature1[:200,:], feature2[:200,:], 
                              feature3[:200,:]],axis = 1)
        
    return feature

In [ ]:
all_feature = []
all_label = []

for i in range(len(file)):
    path = os.path.join(path_dir, file[i])
    name = os.listdir(os.path.join(path_dir, file[i]))
    for j in range(len(name)):
        try:
            feature = raw_features(os.path.join(path, name[j]))
        except:
            print("{}  {} 文件存在异常".format(path, name[j]))
        else:
            all_feature.append(feature)
            all_label.append(label[i])
            print("shape of {}  {} is {}".format(path, name[j], feature.shape))
            print("label is {}", label[i])

In [6]:
path = r"C:\Users\sun\Desktop\models\features\MODMA\all_data_25s"
np.save(os.path.join(path, 'all_features'), all_feature)
np.save(os.path.join(path, 'all_label'), all_label)

In [7]:
print(len(all_feature))
print(len(all_label))

1503
1503


In [8]:
all_data = []
for i in range(len(all_feature)):
    all_data.append(all_feature[i][0:200, :])

In [9]:
path = r"C:\Users\sun\Desktop\models\features\MODMA"
np.save(os.path.join(path, 'all_data_MODMA'), all_data)
np.save(os.path.join(path, 'all_label_MODMA'), all_label)